### Notebook to test conv/strat classification algorithm.

Origins:
- Steiner et al. (1995)
- Rogers (2010)
- Rios-Berrios (2020) f90 algorithm

James Ruppert  
jruppert@ou.edu  
11/25/22

In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib
from matplotlib import ticker
import matplotlib.pyplot as plt
import subprocess
import wrf
from scipy.ndimage import uniform_filter

#### Directories and settings

In [2]:
storm = 'haiyan'
main = "/ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/wrfenkf/"
figdir = "/home/jamesrup/figures/tc/ens/"+storm+'/'

# Time selection
it_read = 37
hr_tag = str(np.char.zfill(str(it_read), 2))

# Simulation and test to read
memb = 'memb_01'
test = 'ctl'

datdir = main+storm+'/'+memb+'/'+test+'/'

#### Read variables

In [3]:
# List of WRF output files
process = subprocess.Popen(['ls '+datdir+'wrfout_d02_*'],shell=True,
    stdout=subprocess.PIPE,universal_newlines=True)
# find desired time
for it in range(it_read): output = process.stdout.readline()

wrffil_path = output.strip() #[3]
wrfnc = Dataset(wrffil_path)

z = wrf.getvar(wrfnc,'z',units='m')
dbz = wrf.getvar(wrfnc,'dbz')
w = wrf.getvar(wrfnc,'wa')

dx = wrfnc.DX*1e-3
dy = wrfnc.DY*1e-3

wrfnc.close()

#### Vertical interpolation

In [4]:
z1 = 400 # m
z2 = 1000 # m
dbz_z = wrf.interplevel(dbz,z,(z1,z2))
w_z = wrf.interplevel(w,z,(z1,z2))

#### Classification algorithm function

rainclass = classify(dims, dx, dbz, w)

Assumes dx = dy

INPUT:
- dims: tuple as dims=(nz,nx1,nx2)
- dx1: delta-x1 in km
- dx2: delta-x2 in km
- dbz: radar reflectivity factor (dBZ) as f(z,x1,x2)
- w: vertical motion (m/s) as f(z,x1,x2)

RETURN:
- classification as f(x1,x2), where
    - 0 = non-raining
    - 1 = conv
    - 2 = strat
    - 3 = other

In [11]:
def classify(dims, dx, dbz, w):

    # Peakedness criteria
    def deltadbz(dbz):
      if dbz <= 0:
        deltadbz=10
      else:
        # if dbz < 42.43: # Steiner
        if dbz < 45: # Rogers
          # deltadbz = 10 - ((dbz**2)/180) # Steiner
          deltadbz = 15 - ((dbz**2)/135) # Rogers
        else:
          deltadbz = 0
      return deltadbz

    nz,nx1,nx2 = dims[0],dims[1],dims[2]

    w_mean = np.mean(w, axis=0)

    rainclass=np.zeros((nx1,nx2))

    for ik in range(nz):

        dbz_ik = np.copy(dbz[ik,:,:])
        dbz_ik[np.where(~np.isfinite(dbz_ik))] = -9999

        # Background reflectivity
        xcheck = 11 # km; Steiner et al.
        npts_radius = int(np.round(xcheck/dx))
        dbz_bg = uniform_filter(dbz_ik, size=npts_radius, mode='nearest')

        # First pass: dBZ > 45 = convective 
        ind = np.where(dbz_ik > 45)
        count = ind[0].size
        rainclass[ind] = 1

        ind_le45 = np.where(dbz_ik <= 45)
        count = ind_le45[0].size
        if count == 0: continue

        # Second pass: if averaged w < 0.1 m/s then it's unlikely to be convective
        # ind_wlt = np.where(w_mean < 0.5)
        ind_wgt = np.where(w_mean[ind_le45] >= 0.5)
        count = ind_wgt[0].size
        if count == 0: continue

        # Background check
        # delta = deltadbz(dbz_ik[ind_le45[ind_wgt]])
        # print(delta)
        # # diff = dbz

    return rainclass

In [6]:
# dbz_filt = np.copy(dbz_z)
#     # dbz[np.where(~np.isfinite(dbz))] = -9999
# dbz_filt[np.where(~np.isfinite(dbz_z))] = -9999
# test_filt = uniform_filter(dbz_filt[0,:,:], size=(4,4), mode='nearest')
# print(dbz_filt[0,200,0:20])
# print()
# print(test_filt[200,0:20])

In [12]:
nx = dbz_z.west_east.size
ny = dbz_z.south_north.size
nz = dbz_z.level.size
dims = (nz,ny,nx)

rainclass = classify(dims, dx, dbz_z, w_z)

# print(np.max(rainclass),np.min(rainclass))
# print(dbz_z[0,600:,200:])

MemoryError: Unable to allocate 3.89 TiB for an array with shape (1033437, 1033437) and data type float32